In [1]:
from notebooks.ford.vins import search_vins

vins = await search_vins(searches=None, include_openpilot=False)

Loaded AsBuilt data for 36746 VINs
Loaded 36746 VINs (filter_comment=None, include_openpilot=False, skipped=181, missing_asbuilt=0)


In [2]:
import pandas as pd

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.ecu import FordEcu
from panda.python.uds import DATA_IDENTIFIER_TYPE

ECUS = [
  # FordEcu.AntiLockBrakeSystem,
  # FordEcu.CruiseControlModule,
  FordEcu.ImageProcessingModuleA,
  # FordEcu.PowerSteeringControlModule,
]

DID_PART_NUMBER = 0xF111

def print_identifiers(ecu):
  rows = set()
  for vin in vins:
    data = AsBuiltData.from_vin(vin)
    part = data.get_identifier(ecu, DID_PART_NUMBER)
    fw = data.get_identifier(ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
    rows.add((part, fw))
  return pd.DataFrame(rows, columns=['part', 'fw'])

for ecu in ECUS:
  print(f'{ecu.name} ({hex(ecu)})')
  print(print_identifiers(ecu))
  print()

ImageProcessingModuleA (0x706)
                part               fw
0     MR3T-14F403-CA   MR3T-14F397-AE
1    ML3T-14H107-FDA  ML3T-14H102-ABS
2     LJ6T-14F403-CA   LJ6T-14F397-AE
3     PC3T-14H107-EE  PC3T-14H102-ABK
4     LL1T-14F403-CA   LL1T-14F397-AC
..               ...              ...
194  PJ6T-14H107-EDB  PJ6T-14H102-ABL
195   NK3T-14F403-CA   NK3T-14F397-AB
196  PJ6T-14H107-ECD  PJ6T-14H102-ABS
197   GJ7T-14G025-NB   GJ7T-14G019-NB
198   JL3T-14F403-AB   JL3T-14F397-AD

[199 rows x 2 columns]



In [3]:
def parse_part_number(part_number):
  prefix, core, suffix = part_number.split('-')
  return {
    'prefix': prefix,
    'core': core,
    'suffix': suffix,
  }

def print_core(ecu):
  rows = set()
  for vin in vins:
    data = AsBuiltData.from_vin(vin)
    part = data.get_identifier(ecu, DID_PART_NUMBER)
    if part:
      parts = parse_part_number(part)
      part = parts['core']
    rows.add(part)
  return pd.DataFrame(rows)

for ecu in ECUS:
  print(f'{ecu.name} ({hex(ecu)})')
  print(print_core(ecu))
  print()

ImageProcessingModuleA (0x706)
        0
0  14G025
1    None
2  14H107
3  14F403



In [4]:
def print_configuration_size(ecu):
  rows = []
  for vin in vins:
    data = AsBuiltData.from_vin(vin)
    part = data.get_identifier(ecu, DID_PART_NUMBER)
    if not part:
      continue
    config = data.get_configuration(ecu)
    if not config:
      rows.append({'part': part})
      continue
    blocks = dict({str(block_no + 1): len(block) for block_no, block in enumerate(config)})
    blocks['part'] = part
    rows.append(blocks)

  df = pd.DataFrame.from_records(rows)
  df.drop_duplicates(inplace=True, ignore_index=True)
  for col in df.columns:
    if col == 'part':
      continue
    df[col] = df[col].astype('Int64')

  df = df.join(df['part'].apply(parse_part_number).apply(pd.Series))
  df.drop(columns=['part'], inplace=True)

  df = df.reindex(sorted(df.columns), axis=1)
  df.fillna(0, inplace=True)
  return df

for ecu in ECUS:
  print(f'{ecu.name} ({hex(ecu)})')
  df = print_configuration_size(ecu)
  # print(df)

  constants = dict()
  for col in df.columns:
    if col in ('prefix', 'core', 'suffix'):
      continue
    if df[col].nunique() == 1:
      constants[col] = df[col].iloc[0]
  print(f'constants: {constants}')
  df.drop(columns=constants.keys(), inplace=True)

  print()

  # try grouping by everything EXCEPT part number
  for part, group in df.groupby([col for col in df.columns if col not in ('prefix', 'core', 'suffix')]):
    print(part)
    print(group)
    print()

  print()
  print()
  print()

ImageProcessingModuleA (0x706)
constants: {}

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
     1  10  11  12  13  14  15  16  17  18  ...  38  4  5  6  7  8  9    core  \
151  0   0   0   0   0   0   0   0   0   0  ...   0  0  0  0  0  0  0  14F403   
165  0   0   0   0   0   0   0   0   0   0  ...   0  0  0  0  0  0  0  14F403   

     prefix  suffix  
151    LJ7T      CA  
165    LC5T      FA  

[2 rows x 41 columns]

(4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0)
    1  10  11  12  13  14  15  16  17  18  ...  38  4  5  6  7  8  9    core  \
0   4   0   0   0   0   0   0   0   0   0  ...   0  4  0  0  0  0  0  14G025   
92  4   0   0   0   0   0   0   0   0   0  ...   0  4  0  0  0  0  0  14G025   

    prefix  suffix  
0     EG1T      KF  
92    DA5T      CM  

[2 rows x 41 columns]

(8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 